In [11]:
# NOTE: you CAN change this cell
# If you want to use your own database, download it here
# !gdown ...

!gdown --fuzzy https://drive.google.com/file/d/1uxbTTuIi5PndOCFBa3g6ZxRjt8gMwhWl/view?usp=sharing
!gdown --fuzzy https://drive.google.com/file/d/1O6QJinc8zpS1s__2L2NlmOZ_DLJTPX6O/view?usp=sharing
!gdown --fuzzy https://drive.google.com/file/d/1BLVpAjqzwUZUcum8E5w6CcXjx2eL3etA/view?usp=sharing
!gdown --fuzzy https://drive.google.com/file/d/1uZqPzXc97RDt3yRIb80xer_EgWXc9BnO/view?usp=sharing

Downloading...
From: https://drive.google.com/uc?id=1uxbTTuIi5PndOCFBa3g6ZxRjt8gMwhWl
To: /content/database.csv
100% 369k/369k [00:00<00:00, 6.61MB/s]
Downloading...
From: https://drive.google.com/uc?id=1O6QJinc8zpS1s__2L2NlmOZ_DLJTPX6O
To: /content/list_district.txt
100% 8.41k/8.41k [00:00<00:00, 25.0MB/s]
Downloading...
From: https://drive.google.com/uc?id=1BLVpAjqzwUZUcum8E5w6CcXjx2eL3etA
To: /content/list_province.txt
100% 813/813 [00:00<00:00, 3.47MB/s]
Downloading...
From: https://drive.google.com/uc?id=1uZqPzXc97RDt3yRIb80xer_EgWXc9BnO
To: /content/list_ward.txt
100% 95.2k/95.2k [00:00<00:00, 5.22MB/s]


In [2]:
# NOTE: you CAN change this cell
# Add more to your needs
# you must place ALL pip install here
# !pip install editdistance
!pip install -q pandas

In [5]:
# NOTE: you CAN change this cell
# import your library here
import time
import numpy as np
import pandas as pd
import re

In [23]:
# NOTE: you MUST change this cell
# New methods / functions must be written under class Solution.
class Solution:
    def __init__(self):
        # list provice, district, ward for private test, do not change for any reason

        self.province_path = 'list_province.txt'
        self.district_path = 'list_district.txt'
        self.ward_path = 'list_ward.txt'

        # write your preprocess here, add more method if needed
        def read_text_db(self, file_path):
            lines_list = []
            with open(file_path, "r", encoding="utf-8") as file:
                lines_list = file.readlines()
            lines_list = [line.strip() for line in lines_list]
            return lines_list
        self.list_province = read_text_db(self, self.province_path)
        self.list_district = read_text_db(self, self.district_path)
        self.list_ward = read_text_db(self, self.ward_path)
        self.df_address = pd.read_csv('database.csv')
        self.df_address = self.df_address.fillna('')
        self.df_address['ward_clean'] = self.df_address['ward'].str.lower().str.replace(r'[^0-9a-zàáâãèéêìíòóôõùúýăđĩũơưạảấầẩẫậắằẳẵặẹẻẽếềểễệỉịọỏốồổỗộớờởỡợụủứừửữựỳỵỷỹ]', '', regex=True)
        self.df_address['district_clean'] = self.df_address['district'].str.lower().str.replace(r'[^0-9a-zàáâãèéêìíòóôõùúýăđĩũơưạảấầẩẫậắằẳẵặẹẻẽếềểễệỉịọỏốồổỗộớờởỡợụủứừửữựỳỵỷỹ]', '', regex=True)
        self.df_address['province_clean'] = self.df_address['province'].str.lower().str.replace(r'[^0-9a-zàáâãèéêìíòóôõùúýăđĩũơưạảấầẩẫậắằẳẵặẹẻẽếềểễệỉịọỏốồổỗộớờởỡợụủứừửữựỳỵỷỹ]', '', regex=True)

        self.df_address['ward_district'] = (self.df_address['ward_clean'] + self.df_address['district_clean'])
        self.df_address['ward_province'] = (self.df_address['ward_clean'] + self.df_address['province_clean'])
        self.df_address['district_province'] = (self.df_address['district_clean'] + self.df_address['province_clean'])
        self.df_address['address'] = (self.df_address['ward_clean'] + self.df_address['district_clean'] + self.df_address['province_clean'])

        self.df_address['bigram_ward'] = [set(row[i:i+2] for i in range(len(row) - 1)) for row in self.df_address['ward_clean']]
        self.df_address['bigram_district'] = [set(row[i:i+2] for i in range(len(row) - 1)) for row in self.df_address['district_clean']]
        self.df_address['bigram_province'] = [set(row[i:i+2] for i in range(len(row) - 1)) for row in self.df_address['province_clean']]

        self.df_address['bigram_ward_district'] = [set(row[i:i+2] for i in range(len(row) - 1)) for row in self.df_address['ward_district']]
        self.df_address['bigram_ward_province'] = [set(row[i:i+2] for i in range(len(row) - 1)) for row in self.df_address['ward_province']]
        self.df_address['bigram_district_province'] = [set(row[i:i+2] for i in range(len(row) - 1)) for row in self.df_address['district_province']]
        self.df_address['bigram_db'] = [set(row[i:i+2] for i in range(len(row) - 1)) for row in self.df_address['address']]
        pass

    def sorensen_dice_coefficient(self, bigrams1, bigrams2):
        """Calculate Sørensen-Dice similarity between two bigram sets."""
        # intersection = len(bigrams1.intersection(bigrams2))
        intersection = len(bigrams1 & bigrams2)
        size1 = len(bigrams1)
        size2 = len(bigrams2)
        return (2 * intersection) / (size1 + size2) if size1 + size2 > 0 else 0.0

    def process(self, s: str):
        # write your process string here
        _input = re.sub(r'[^0-9a-zàáâãèéêìíòóôõùúýăđĩũơưạảấầẩẫậắằẳẵặẹẻẽếềểễệỉịọỏốồổỗộớờởỡợụủứừửữựỳỵỷỹ]', '', s.lower())
        bigram_input = set(_input[i:i+2] for i in range(len(_input) - 1))
        self.df_address['similarity'] = [self.sorensen_dice_coefficient(bigram_input, row) for row in self.df_address['bigram_db']]
        index_max_coef = self.df_address['similarity'].idxmax()

        result_ward = self.df_address.iloc[index_max_coef,0]
        result_district = self.df_address.iloc[index_max_coef,1]
        result_province = self.df_address.iloc[index_max_coef,2]

        similarity_ward_district = self.sorensen_dice_coefficient(self.df_address.iloc[index_max_coef]['bigram_ward_district'], bigram_input)
        similarity_ward_province = self.sorensen_dice_coefficient(self.df_address.iloc[index_max_coef]['bigram_ward_province'], bigram_input)
        similarity_district_province = self.sorensen_dice_coefficient(self.df_address.iloc[index_max_coef]['bigram_district_province'], bigram_input)
        similarity_address = self.df_address.iloc[index_max_coef]['similarity']

        max_similarity = max(similarity_ward_province, similarity_district_province, similarity_address)
        if similarity_district_province >= max_similarity:
            result_ward = ""
        elif similarity_ward_province >= max_similarity:
            result_district = ""
        elif similarity_ward_district >= max_similarity:
            result_province = ""

        return {
            "province": result_province,
            "district": result_district,
            "ward": result_ward,
        }

In [14]:
# NOTE: DO NOT change this cell
# This cell is for downloading private test
!rm -rf test.json
# this link is public test
!gdown --fuzzy https://drive.google.com/file/d/1PBt3U9I3EH885CDhcXspebyKI5Vw6uLB/view?usp=sharing -O test.json

Downloading...
From: https://drive.google.com/uc?id=1PBt3U9I3EH885CDhcXspebyKI5Vw6uLB
To: /content/test.json
100% 79.4k/79.4k [00:00<00:00, 4.42MB/s]


In [26]:
# NOTE: DO NOT change this cell
# This cell is for scoring

TEAM_NAME = 'TEAM_2'  # This should be your team name
EXCEL_FILE = f'{TEAM_NAME}.xlsx'

import json
import time
with open('test.json') as f:
    data = json.load(f)

summary_only = True
df = []
solution = Solution()
timer = []
correct = 0
for test_idx, data_point in enumerate(data):
    address = data_point["text"]

    ok = 0
    try:
        start = time.perf_counter_ns()
        result = solution.process(address)
        answer = data_point["result"]
        finish = time.perf_counter_ns()
        timer.append(finish - start)
        ok += int(answer["province"] == result["province"])
        ok += int(answer["district"] == result["district"])
        ok += int(answer["ward"] == result["ward"])
        df.append([
            test_idx,
            address,
            answer["province"],
            result["province"],
            int(answer["province"] == result["province"]),
            answer["district"],
            result["district"],
            int(answer["district"] == result["district"]),
            answer["ward"],
            result["ward"],
            int(answer["ward"] == result["ward"]),
            ok,
            timer[-1] / 1_000_000_000,
        ])
    except Exception as e:
        df.append([
            test_idx,
            address,
            answer["province"],
            "EXCEPTION",
            0,
            answer["district"],
            "EXCEPTION",
            0,
            answer["ward"],
            "EXCEPTION",
            0,
            0,
            0,
        ])
        # any failure count as a zero correct
        pass
    correct += ok


    if not summary_only:
        # responsive stuff
        print(f"Test {test_idx:5d}/{len(data):5d}")
        print(f"Correct: {ok}/3")
        print(f"Time Executed: {timer[-1] / 1_000_000_000:.4f}")


print(f"-"*30)
total = len(data) * 3
score_scale_10 = round(correct / total * 10, 2)
if len(timer) == 0:
    timer = [0]
max_time_sec = round(max(timer) / 1_000_000_000, 4)
avg_time_sec = round((sum(timer) / len(timer)) / 1_000_000_000, 4)

import pandas as pd

df2 = pd.DataFrame(
    [[correct, total, score_scale_10, max_time_sec, avg_time_sec]],
    columns=['correct', 'total', 'score / 10', 'max_time_sec', 'avg_time_sec',],
)

columns = [
    'ID',
    'text',
    'province',
    'province_student',
    'province_correct',
    'district',
    'district_student',
    'district_correct',
    'ward',
    'ward_student',
    'ward_correct',
    'total_correct',
    'time_sec',
]

df = pd.DataFrame(df)
df.columns = columns

print(f'{TEAM_NAME = }')
print(f'{EXCEL_FILE = }')
print(df2)

!pip install xlsxwriter
writer = pd.ExcelWriter(EXCEL_FILE, engine='xlsxwriter')
df2.to_excel(writer, index=False, sheet_name='summary')
df.to_excel(writer, index=False, sheet_name='details')
writer.close()


------------------------------
TEAM_NAME = 'TEAM_2'
EXCEL_FILE = 'TEAM_2.xlsx'
   correct  total  score / 10  max_time_sec  avg_time_sec
0     1217   1350        9.01        0.0536        0.0146
